In [1]:
import pandas as pd
import numpy as np
import os
import datetime
import math
import json
import torch
import random
from sklearn.model_selection import train_test_split
from spotlight.interactions import Interactions
from sklearn.metrics.pairwise import cosine_similarity
from spotlight.factorization.implicit import ImplicitFactorizationModel

In [579]:
import numpy as np
import json
import os

data_path = '/Users/haoxinli/Downloads/mpd.v1/data/'
playlist_folder = os.listdir(data_path)

## Preparing data

CF_baseline_Train = []
CF_baseline_Val = []

counter = 0
for i in range(50):
    with open(data_path+playlist_folder[i]) as f:
        raw_data = json.load(f)

    playlists = raw_data['playlists']

    for playlist in playlists:
        pid = playlist['pid']
        for track in playlist['tracks']:
            track_uri = track['track_uri']
            rand = random.random()
            if rand < 0.8:
                CF_baseline_Train.append([pid,track_uri])
            else:
                CF_baseline_Val.append([pid,track_uri])
    counter += 1

with open("CF_baseline_Train_40000.json", "w") as f:
    data_json = json.dump(CF_baseline_Train,f)
    
with open("CF_baseline_Val_10000.json", "w") as f:
    data_json = json.dump(CF_baseline_Val,f)

In [2]:
## Load data

with open('CF_baseline_Train_40000.json', 'r') as f:
    data = json.load(f)

users = [user for user, song in data]
item = [song for user, song in data]

## Assign id to each playlist title

song_per_user = {}
for user in users:
    if user not in song_per_user.keys():
        song_per_user[user] = 1
    else:
        song_per_user[user] += 1
# print(len(song_per_user))


pids = {}
num = 0
for user in song_per_user.keys():
    pids[user] = num
    num += 1
# print(len(pids))
    
    
users_transform = []
for user in users:
    users_transform.append(pids[user])
# print(len(users_transform))
    
    
## Assign id to each track id
# appearance of each track
count = {}
for i in item:
    if i not in count.keys():
        count[i] = 1
    else:
        count[i] = count[i] + 1
# print(len(count))
        
        
item_id = {}
id = 0
for i in count.keys():
    item_id[i] = id
    id = id+1
# print(len(item_id))


item_transformed = []
for i in item:
    item_transformed.append(item_id[i])
# print(len(item_transformed))
    
id_to_track = [0] * len(item_id.keys())
for item in item_id.keys():
    id_to_track[item_id[item]] = item
# print(len(id_to_track))

ratings = np.ones(len(item_transformed))

data = Interactions(np.array(users_transform), np.array(item_transformed), ratings)

## Baseline Model: Collaborative Filtering
#### Recommend songs based on other similar playlists, evaulated using R-Precision score

In [536]:
model = ImplicitFactorizationModel(n_iter = 1)
model.fit(data, verbose = 1)
torch.save(model, 'baseline_model_40000')

In [667]:
model = torch.load('baseline_model_40000')

In [650]:
with open('CF_baseline_Val_10000.json', 'r') as f:
    data = json.load(f)
    
validation = []

playlist_num = data[0][0]
songs = []
for item in data:
    playlist_num_new = item[0]
    if playlist_num_new == playlist_num:
        songs.append(item[1])
        playlist_num = playlist_num_new
    else:
        validation.append(songs)
        songs = []
        songs.append(item[1])
        playlist_num = playlist_num_new


In [1]:
scores = []
users = np.array(range(40000))

for playlist in validation[0:200]:

    a = []
    for item in playlist:
        try:
            a.append(item_id[item])
        except:
            pass
    if len(a) > 3:
        X_train, y_test = train_test_split(a, train_size = 0.7)
    
        dic = {}
        score = np.array([0] * len(model.predict(1)))
        sum_cos = 0

        inp = np.array(X_train)
        random.shuffle(users)
        users = users[0:100]
        for i in users:
            s = model.predict(i, np.array(inp))
            cos_sim = cosine_similarity(np.array([s]),np.array([inp]))
            sum_cos = sum_cos + cos_sim
            score = cos_sim * model.predict(i) + score
        score = score / sum_cos

        for index in np.argsort(score[0])[-1:-251:-1]:
            dic[id_to_track[index]] = score[0][index]
        ans = []

        for item in dic:
            ans.append(item_id[item])

        count = 0
        for song in y_test:
            if song in ans:
                count += 1 
        scores.append(count/len(y_test))
        print(np.array(scores).mean())

In [589]:
r_score = np.array(scores).mean()
r_score

0.007930596224636221